# Advanced Coding Group Project - 2024/25 
## Students: Michele Turco, Mattia Cervelli, Lorenzo Laterza 

### Task 1

Setup your python environment and download the credit-score data

In [1]:
# Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# Read the csv file using pandas
credit_card = pd.read_csv('train_biased.csv')

# Identify Number of Columns and Rows
n_rows = len(credit_card)
n_columns = len(credit_card.columns)
print(f"The dataset contains {n_columns} columns")
print(f"The dataset contains {n_rows} columns")

The dataset contains 27 columns
The dataset contains 100000 columns


## 2) EDA